
# Hahow 課程資料爬蟲分析
透過 Selenium 與 BeautifulSoup 爬取 Hahow 平台上各類別的課程資訊，輸出為 CSV 檔供後續資料分析使用。



### **📌專案目的**

模擬使用者在網站上的瀏覽與選課行為，抓取每一類別下的所有課程標題、熱門標籤與連結。

### **📋 使用工具**

- Python
- Selenium
- BeautifulSoup
- Pandas
- ChromeDriver

### **🔧 爬蟲流程介紹**


#### **1. 建立課程分類與對應網址代碼字典**
   - 將 Hahow 平台的課程分類轉換為可用於爬蟲的網址參數
#### **2. 設定 User-agent 與 Proxy**
   - 模擬一般使用者行為，降低被封鎖的風險
#### **3. 使用 Selenium 開啟課程分類頁面**
   - 自動化瀏覽器操作以取得完整頁面內容
#### **4. 使用 BeautifulSoup 解析課程清單區塊**
   - 提取 HTML 中每門課程的基本資訊
#### **5. 撰寫自定義函數處理單一課程資料**
   - 擷取課程名稱、熱門標籤、連結共三個欄位
#### **6. 將結構化資料存為 CSV 檔案**
   - 每分類每頁資料儲存於 `scraped_data/`資料夾，便於後續分析
#### **7. 加入例外處理與計時功能**
   - 確保流程穩定，並評估整體執行效能

### **⚠️ 爬蟲注意事項**

- **本程式中的代理 IP 與 User-Agent 僅為示範用途**，實際使用時請更換為有效的 Proxy 與 User-Agent，請注意免費代理有可能失效，請定期更新以避免請求失敗或遭平台封鎖。
- **Hahow 頁面為動態生成，課程區塊的 `class` 名稱（如 `gtWJhG`）可能會變動**，若無法擷取資料請參考原始碼更新 class。

#### 模組匯入區

In [9]:
# 匯入基本模組與爬蟲相關工具

import os
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service #路徑


#### 自訂函數區

In [10]:
# 自定義函數：處理每一筆課程資料，擷取課程名稱、熱門標籤與連結

def course_item(data):
    course_list = [] # 用來儲存處理後的課程資料

    for item in data:
        
        # 擷取課程名稱並去除多餘空白
        text = item.text.strip() 

        # 尋找課程的連結標籤，若有則提取連結，若無則顯示為「無連結」
        link_tag = item.find("a") 
        link = link_tag.get("href") if link_tag else "無連結"  

        # 尋找熱門標籤所在的 div，若有則提取文字，否則顯示為「無標籤」
        popular_tag = item.find("div","sc-1x67v6w-0 sc-1x67v6w-1 jYbAhu icUvip")
        popular = popular_tag.text.strip() if popular_tag else "無標籤"

        # 將每筆課程資料加入清單
        course_list.append([text, popular, link])

    return course_list # 回傳所有課程資料


#### 程式設定區

In [11]:
# 定義課程類別與對應網址代碼

course_type = {
    "音樂": "music", "語言": "language", "攝影": "photography", "藝術": "art",
    "設計": "design", "人文": "humanities", "行銷": "marketing",
    "程式": "programming", "投資理財": "finance-and-investment",
    "職場技能": "career-skills", "手作": "handmade", "生活品味": "lifestyle"
}

# 設定 User-Agent 清單 "音樂": "music","人文":"humanities",
user_agents_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:110.0) Gecko/20100101 Firefox/110.0"
]

# 設定 Proxies 清單 
proxies_list = [
    "41.223.119.156:3128"
]

# 隨機選取一個 User-Agent 和 proxy，降低被網站偵測為機器人的風險
user_agent = random.choice(user_agents_list)
proxy = random.choice(proxies_list)

# 設定 Chrome 瀏覽器選項
chrome_options = Options()
chrome_options.add_argument(f"user-agent={user_agent}")
chrome_options.add_argument(f"--proxy-server=http://{proxy}")

# 設定 chromedriver 路徑
service = Service(executable_path="./chromedriver.exe")


#### 爬蟲主程式區

In [12]:
# 建立資料夾
folder_name = "scraped_data"

# 如果資料夾不存在就新建立
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print("已建立scraped_data資料夾")


# 開始計時並啟動webdriver
start_time = time.time()
driver = webdriver.Chrome(service=service,options=chrome_options)

try:
    # 瀏覽每個課程分類，逐頁爬取課程資料
    for category_name, catagory_code in course_type.items():
        page_number = 1 
        print(f"開始進行{category_name}")

        while True:
            retries = 0 # 每一頁重試次數歸零
            page_success = False
            data = None

            while retries < 3:
            
                try:
                    # 進入目前頁面的 URL 並進入
                    current_url = f"https://hahow.in/group/{catagory_code}?filter=PUBLISHED&page={page_number}"
                    driver.get(current_url)

                    # 等待頁面載入課程資料
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located(
                        (By.XPATH, "//*[@id='main-screen']/section[1]/div[5]/ul")
                    ))

                    # 滾動到底部，確保內容完整載入
                    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

                    # 取得目前頁面 HTML 並使用 BeautifulSoup 解析
                    html = driver.page_source
                    bs = BeautifulSoup(html, "lxml")

                    # 擷取所有課程區塊
                    data = bs.find_all("div", "sc-1awt4tz-1 gtWJhG")

                    # 若無課程資料，表示已到最後一頁，結束爬取
                    if not data:
                        print(f"{category_name} 已爬取完畢，共爬取到第 {page_number - 1} 頁，進行下一個分類")
                        break

                    else:
                        # 呼叫自訂函數處理課程資料，轉為 DataFrame
                        course_data = course_item(data)
                        course_df = pd.DataFrame(course_data, columns=["課程名稱", "熱門標籤", "link"])

                        # 建立檔案路徑
                        filename = os.path.join(folder_name,f"hahow_{category_name}_page{page_number}.csv")

                        # 儲存 CSV 檔案
                        course_df.to_csv(filename, encoding="utf-8-sig", index=False)
                        print(f"第{page_number}頁爬取成功，並輸出csv完成")

                        # 前往下一頁
                        page_success = True
                        page_number += 1

                except Exception as e:
                    retries += 1
                    print(f"{category_name}第{page_number}頁發生錯誤並重試{retries}次:", e)
                    time.sleep(random.uniform(1, 1.5))
            
            if not page_success:
               print(f"⛔ {category_name}第{page_number}頁失敗 3 次，跳過此頁")
               page_number += 1
                
            if not data:
                break
                
finally:
    driver.quit()
    print("全部爬取完成,並關閉瀏覽器")

# 計時結束
end_time = time.time()
execution_time = end_time - start_time
print("耗時{}".format(execution_time))


已建立scraped_data資料夾
開始進行手作
第1頁爬取成功，並輸出csv完成
第2頁爬取成功，並輸出csv完成
手作 已爬取完畢，共爬取到第 2 頁，進行下一個分類
開始進行音樂
第1頁爬取成功，並輸出csv完成
第2頁爬取成功，並輸出csv完成
音樂 已爬取完畢，共爬取到第 2 頁，進行下一個分類
開始進行攝影
第1頁爬取成功，並輸出csv完成
第2頁爬取成功，並輸出csv完成
攝影第3頁發生錯誤並重試1次: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF78615AD02+56930]
	(No symbol) [0x00007FF7860CF602]
	(No symbol) [0x00007FF785F842E5]
	(No symbol) [0x00007FF785FC98ED]
	(No symbol) [0x00007FF785FC9A2C]
	(No symbol) [0x00007FF78600A967]
	(No symbol) [0x00007FF785FEBCDF]
	(No symbol) [0x00007FF7860081E2]
	(No symbol) [0x00007FF785FEBA43]
	(No symbol) [0x00007FF785FBD438]
	(No symbol) [0x00007FF785FBE4D1]
	GetHandleVerifier [0x00007FF7864D6F8D+3711213]
	GetHandleVerifier [0x00007FF7865304CD+4077101]
	GetHandleVerifier [0x00007FF78652865F+4044735]
	GetHandleVerifier [0x00007FF7861F9736+706710]
	(No symbol) [0x00007FF7860DB8DF]
	(No symbol) [0x00007FF7860D6AC4]
	(No symbol) [0x00007FF7860D6C1C]
	(No symbol) [0x00007FF7860C68D4]
	BaseThreadInitThunk [0x00007FFDFE277374+20]
	RtlUser